### Extraindo dados de boletos salvos em uma pasta

Este notebook realiza a leitura de boletos de pagamentos salvos em uma pasta e a extração de dados necessários para uma análise. 
Além disso, salva os dados extraídos em um dataset no qual serão imputados os dados dos novos boletos que forem salvos nessa respectiva pasta

In [2]:
#importando as bibliotecas necessárias
import pandas as pd
import glob
import fitz

#listando todos os boletos localizados na pasta
pasta = 'boleto'
caminho_boleto = glob.glob(f"{pasta}/*.pdf")

boletos_lidos=0

#lendo cada boleto salvo na pasta
for boleto in caminho_boleto:
    with fitz.open(boleto) as doc:
        num_pages = doc.page_count
        for page_number in range(num_pages):
            page = doc.load_page(page_number)
            text = page.get_text()
            parts = text.split("\\")
            elements=[]

            for i,part in enumerate(parts):
                elements.append(part)

                if i <len(parts)-1:
                    elements[-1]+="\\"
        
        boletos_lidos+=1

    sequencia_inicial = elements[0]
    lista_palavras = sequencia_inicial.split()


    df=pd.DataFrame({'Campo':lista_palavras})
    df.index.name = 'Posição'

    #extraindo as informações necessárias
    banco=lista_palavras[1]
    numero = lista_palavras[36]
    cnpj=lista_palavras[25]
    data = lista_palavras[32]
    codigo_barra=lista_palavras[294]+lista_palavras[295]+lista_palavras[296]+lista_palavras[297]+lista_palavras[298]
    valor = lista_palavras[353]
    vencimento = lista_palavras[300]
    cod_pagador=lista_palavras[416]
    
    insert_list = [banco,numero,cnpj,data,codigo_barra,valor,vencimento,cod_pagador]
    base_carga = pd.DataFrame(columns=['cod_banco','cod_boleto','cnpj','data','cod_barra','valor','vencimento','cpf'])

    #inserindo as informações de cada boleto para carregar na base
    for i,coluna in enumerate(base_carga.columns):
        base_carga.at[0,coluna]=insert_list[i]

    #atualizando base 
    base_hist = pd.read_csv('hist/base.csv',sep=';',header=0)

    base_atualiza = pd.concat([base_hist,base_carga],ignore_index=True)
    base_atualiza.drop_duplicates().to_csv('hist/base.csv',index=False,sep=';')
    